# <a id="1"></a>
# 
# <h1 style="background-color:#33E4E2;font-family:newtimeroman;font-size:220%;text-align:center;border-radius:#33E4E2 12px 30px;">👀Covid 19 has impacted most of everyones lives over the past 2 years. While meandering around the internet looking for an interesting read, something caught my eye. Atlanta's Hartsfield-Jackson Atlanta International Airport(has been consistantly one of the busiest airports in the world) has taken a massive hit in airport traffic. This lead me here, to kaggle, checking out some airport data. Lets take a look and see what we can come up with

In [ ]:
from datetime import date
import os

import geopandas as gpd
import folium
import geoplot as gplt
import mapclassify
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np ## Linear Alegbra
import pandas as pd ## data processing, file I/O
import plotly.express as px
import re
import seaborn as sns
from shapely.geometry import Point, Polygon
from shapely.geometry import MultiPolygon

In [ ]:
%matplotlib inline

In [ ]:
airport_impact = pd.read_csv("../input/covid19s-impact-on-airport-traffic/covid_impact_on_airport_traffic.csv")

📌Taking a quick peak at the data before we get in to preprocessing

In [ ]:
airport_impact.head()

📌Lets check for any missing values

In [ ]:
airport_impact.info()

📌Here we can get some quick info, easier to pull this from .info rather than -->.head<--. We can see there are 11 columns, 7247 entries, and no missing values(yay).

<a id="1"></a>

<h1 style="background-color:#33E4E2;font-family:newtimeroman;font-size:220%;text-align:center;border-radius:#33E4E2 12px 30px;">👀Preprocessing

In [ ]:
airport_impact['Date'] = airport_impact['Date'].map(lambda x: date.fromisoformat(x))

📌Creating weekdays

In [ ]:
airport_impact['Weekday'] = airport_impact['Date'].map(lambda x: x.weekday())

wk_list = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

airport_impact['Weekday'] = [wk_list[idx] for idx in airport_impact['Weekday']]

In [ ]:
def cut_long(point):
    
    long, _ = point[6:-1].split(" ")
    return float(long)


def cut_lat(point):
    
    _, lat = point[6:-1].split(" ")
    return float(lat)

In [ ]:
airport_impact["long"] = airport_impact['Centroid'].map(cut_long)

airport_impact['lat'] = airport_impact['Centroid'].map(cut_lat)

In [ ]:
drop_columns = [col for col in airport_impact.columns if not col in ['AggregationMethod', 'Version', "Centroid"]]

airport_impact = airport_impact[drop_columns]

In [ ]:
airport_impact.info()

📌Taking a look at our data once more, you can see we are down to 9 columns. Above, we removed Aggregation Method, Version, and Centroid(we replaced it with long and lat).

In [ ]:
airport_impact_unique = airport_impact[~airport_impact[["AirportName"]].duplicated()].reset_index(drop=True)
airport_impact_unique

📌The data is much easier to handle when there arent so many duplicated airport names. Crunching things down can save a lot of time.

In [ ]:
df_geometry = airport_impact_unique[['AirportName', 'City', 'State',
                                    'ISO_3166_2', 'Country', 'lat']]

📌Here We can use Folium to see the airports on a cool map. I love interactive maps.

In [ ]:
def visualize_airport_map(df, zoom):
    
    lat_map = 30.038557
    lon_map = 31.231781
    
    f = folium.Figure(width=1000, height=500)
    m = folium.Map([lat_map,lon_map], zoom_start=zoom).add_to(f)
    
    for i in range(0,len(df)):
        folium.Marker(location=[df['lat'][i],df['long'][i]],icon=folium.Icon(icon_color='Red', icon='plane', prefix='fa')).add_to(m)
                                     
    return m

In [ ]:
airport_impact_map = visualize_airport_map(airport_impact_unique,1)

In [ ]:
airport_impact_map

📌As you can see, almost every single airport we have is located in North America. The only exceptions being Arturo Merino Benítez International Airport in Santiago, Chile, and Inouye International Airport in Honolulu, Hawaii. Since we already know by looking at the map above that most airports are in north america, lets take a look at which airports have the most records in the dataset

In [ ]:
plt.figure(figsize = (10,5))

g = sns.countplot(data = airport_impact, x = "AirportName",
                 order = airport_impact['AirportName'].value_counts().index)

g.set_xticklabels(g.get_xticklabels(), rotation = 90)

g.set_title("Airport Records")

📌We can even go a step further by checking reported for each month.

In [ ]:
df_month = pd.DataFrame(airport_impact["Date"].map(lambda d: d.month).value_counts())

df_month = df_month.reset_index()
df_month = df_month.rename(columns = {'Date':'count', 'index':'month'})

g = sns.barplot(data = df_month.reset_index(), y = 'count', x = 'month')

g.set_xticklabels(g.get_xticklabels(), rotation = 90)
g.set_title('Monthly Labels')

📌And even further, with the weekly records(Thanks to us making this dataframe earlier).

In [ ]:
df_week = pd.DataFrame(airport_impact['Weekday'].value_counts())

g = df_week.plot.pie(y = 'Weekday', figsize = (7,7))

g.set_title("Weekly Records")

<a id="1"></a>

<h1 style="background-color:#33E4E2;font-family:newtimeroman;font-size:220%;text-align:center;border-radius:#33E4E2 12px 30px;">👀Lets check out some mapping and visualization examples using Geopandas.

In [ ]:
gdf = gpd.GeoDataFrame(airport_impact_unique,
                        geometry = gpd.points_from_xy(airport_impact_unique.long,
                                    airport_impact_unique.lat))

In [ ]:
gdf.head()

In [ ]:
world_map = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

world_map.head()

📌Geopandas Example #1
Basic World Map

In [ ]:
g = world_map.plot(color = 'white', edgecolor = 'gray')

g.set_title('World Map')

📌Geopandas Example #2
Map with markers using DF we created earlier.

In [ ]:
ax = world_map.plot(color = 'white', edgecolor = 'gray', figsize = (15, 10))

g = gdf.plot(ax = ax, marker = '*', color = 'maroon', markersize = 50)

g.set_title("Map with Markers")

plt.show()

📌Geopandas Example #3
World map with added Legend and markers.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (15, 10))

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size = '5%', pad = 0.1)

world_map.plot(ax = ax, color = 'white', edgecolor = 'gray')
g = gdf.plot(column = 'PercentOfBaseline', ax = ax, markersize = 50,
            legend = True, cax = cax)
g.set_title("world map with ")

plt.show()

<a id="1"></a>

<h1 style="background-color:#33E4E2;font-family:newtimeroman;font-size:220%;text-align:center;border-radius:#33E4E2 12px 30px;">👀Average weekday airport reports

In [ ]:
df_airport_weekday_mean = airport_impact[['AirportName', 'Weekday', 'PercentOfBaseline']].groupby(['AirportName', 'Weekday']).mean()

df_airport_weekday_mean = df_airport_weekday_mean.reset_index()

df_airport_weekday_mean = df_airport_weekday_mean.merge(df_geometry, on = "AirportName" )

In [ ]:
df_airport_weekday_mean.head()

<a id="1"></a>

<h1 style="background-color:#33E4E2;font-family:newtimeroman;font-size:220%;text-align:center;border-radius:#33E4E2 12px 30px;">👀Plotly Viz

In [ ]:
df_airport_all_mean = airport_impact.groupby(['AirportName']).mean()

df_airport_all_mean = df_airport_all_mean.reset_index()

In [ ]:
df_airport_all_mean.head()

In [ ]:
fig = px.scatter_mapbox(df_airport_all_mean,
                       lat = 'lat',
                       lon = 'long',
                       hover_name = 'AirportName',
                       hover_data = ['PercentOfBaseline'],
                        color = "PercentOfBaseline",
                        zoom = 1,
                        height = 600,
                        size = "PercentOfBaseline",
                        size_max = 30,
                        opacity = 0.5,
                        width = 1300)


fig.update_layout(mapbox_style = 'stamen-terrain')

fig.update_layout(margin = {'r': 0, 't': 0, 'l': 0, 'b': 0})

fig.update_layout(title_text = "Mean of Sunday Traffic")
fig.show()

In [ ]:
fig = px.scatter_mapbox(df_airport_all_mean,
                       lat = 'lat',
                       lon = 'long',
                       hover_name = 'AirportName',
                       hover_data = ['PercentOfBaseline'],
                       color = 'PercentOfBaseline',
                       zoom = 1,
                       height = 600,
                       size = 'PercentOfBaseline',
                       size_max = 30,
                       opacity = 0.5,
                       width = 1300)

fig.update_layout(mapbox_style = 'carto-positron')
fig.update_layout(margin = {'r': 0, 't': 0, 'l': 0, 'b': 0})
fig.show()

<a id="1"></a>

<h1 style="background-color:#33E4E2;font-family:newtimeroman;font-size:220%;text-align:center;border-radius:#33E4E2 12px 30px;">👀This is as far as ive gotten so far. Leave a thumbs up if you enjoy the work!